# Some quick start code for TUM Hackathon

In [2]:
from langchain.document_loaders import PyPDFLoader
from ai_eval.config import global_config as glob

filename = "Allplan_2020_Manual.pdf"

loader = PyPDFLoader(f"{glob.DATA_PKG_DIR}/{filename}")

raw_data = loader.load()

texts = [page.page_content for page in raw_data]

print(f"Number of docs: {len(texts)}")

/Users/avosseler/Github/Team/tum-hackathon/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/Users/avosseler/Github/Team/tum-hackathon/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Number of docs: 319


## (Optional) Preprocess and load data: 

In [3]:
from ai_eval.resources.preprocessor import Preprocessor
from ai_eval.config import global_config as glob

filename = "Allplan_2020_Manual.pdf"

pre = Preprocessor()

docs = pre.fetch_documents(
    blob_path=f"{glob.DATA_PKG_DIR}/{filename}", source="local"
)

documents = pre.chunk_documents(documents=docs)

print(f"Number of processed document chunks: {len(documents)}")

2025-11-21 22:58:30,143 - ai_eval.resources.preprocessor - INFO - Processed /Users/avosseler/Github/Team/tum-hackathon/data/Allplan_2020_Manual.pdf
2025-11-21 22:58:30,151 - ai_eval.resources.preprocessor - INFO - Using SentenceChunker for document chunking.
2025-11-21 22:58:30,151 - ai_eval.resources.preprocessor - INFO - Using SentenceChunker for document chunking.


2025-11-21 22:58:30,143 - ai_eval.resources.preprocessor - INFO - Processed /Users/avosseler/Github/Team/tum-hackathon/data/Allplan_2020_Manual.pdf
2025-11-21 22:58:30,151 - ai_eval.resources.preprocessor - INFO - Using SentenceChunker for document chunking.
2025-11-21 22:58:30,151 - ai_eval.resources.preprocessor - INFO - Using SentenceChunker for document chunking.


Finished 'fetch_documents' in 2.0255 secs


2025-11-21 22:58:30,143 - ai_eval.resources.preprocessor - INFO - Processed /Users/avosseler/Github/Team/tum-hackathon/data/Allplan_2020_Manual.pdf
2025-11-21 22:58:30,151 - ai_eval.resources.preprocessor - INFO - Using SentenceChunker for document chunking.
2025-11-21 22:58:30,151 - ai_eval.resources.preprocessor - INFO - Using SentenceChunker for document chunking.


Finished 'fetch_documents' in 2.0255 secs


🦛 choooooooooooooooooooonk 100% • 319/319 docs chunked [00:02<00:00, 139.47doc/s] 🌱
2025-11-21 22:58:33,800 - ai_eval.resources.preprocessor - INFO - Chunked 319 pages into 319 chunks.

2025-11-21 22:58:33,800 - ai_eval.resources.preprocessor - INFO - Chunked 319 pages into 319 chunks.


2025-11-21 22:58:30,143 - ai_eval.resources.preprocessor - INFO - Processed /Users/avosseler/Github/Team/tum-hackathon/data/Allplan_2020_Manual.pdf
2025-11-21 22:58:30,151 - ai_eval.resources.preprocessor - INFO - Using SentenceChunker for document chunking.
2025-11-21 22:58:30,151 - ai_eval.resources.preprocessor - INFO - Using SentenceChunker for document chunking.


Finished 'fetch_documents' in 2.0255 secs


🦛 choooooooooooooooooooonk 100% • 319/319 docs chunked [00:02<00:00, 139.47doc/s] 🌱
2025-11-21 22:58:33,800 - ai_eval.resources.preprocessor - INFO - Chunked 319 pages into 319 chunks.

2025-11-21 22:58:33,800 - ai_eval.resources.preprocessor - INFO - Chunked 319 pages into 319 chunks.


Finished 'chunk_documents' in 3.6603 secs
Number of processed document chunks: 319


## Get annotated data:

In [4]:
from ai_eval.services.file import JSONService
from ai_eval.config import global_config as glob

json = JSONService(path="generated_qa_data_tum.json", root_path=glob.DATA_PKG_DIR, verbose=True)

qa_data = json.doRead()
print(f"Number of evaluation data samples: {len(qa_data)}")

2025-11-21 22:58:39,823 - ai_eval.services.file - INFO - Read: /Users/avosseler/Github/Team/tum-hackathon/data/generated_qa_data_tum.json


Number of evaluation data samples: 100


### Fit RAG model on the generated data and create evaluation dataset

In [5]:
from ai_eval.resources import eval_dataset_builder as eval

ground_truth_contexts = [item["context"] for item in qa_data]
sample_queries = [item["question"] for item in qa_data]
expected_responses = [item["answer"] for item in qa_data]

Example: using Vertex AI models

In [6]:
from langchain_google_vertexai import ChatVertexAI, VertexAIEmbeddings

chat_model = ChatVertexAI(
                        project=glob.GCP_PROJECT,
                        model_name="gemini-2.5-flash",
                        temperature=0.1,
                        max_retries=2,
                    )

embedding_model = VertexAIEmbeddings(
                        project=glob.GCP_PROJECT,
                        model_name="text-multilingual-embedding-002",
                    )

/Users/avosseler/Github/Team/tum-hackathon/.venv/lib/python3.12/site-packages/google/cloud/aiplatform/models.py:52: FutureWarning: Support for google-cloud-storage < 3.0.0 will be removed in a future version of google-cloud-aiplatform. Please upgrade to google-cloud-storage >= 3.0.0.
  from google.cloud.aiplatform.utils import gcs_utils
/Users/avosseler/Github/Team/tum-hackathon/.venv/lib/python3.12/site-packages/vertexai/_model_garden/_model_garden_models.py:278: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()
/Users/avosseler/Github/Team/tum-hackathon/.venv/lib/python3.12/site-packages/vertexai/_model_garden/_model_garden_models.py:278: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generat

In [ ]:
from langchain_community.vectorstores import FAISS
from ai_eval.resources.rag_template import FAISSRAG, TFIDFRAG

vectorstore = FAISS.from_documents(documents, embedding_model)

# 1. Create your RAG instance
#rag = FAISSRAG(chat_model, documents, k=3, vectorstore=vectorstore)    # some vanilla example
rag = TFIDFRAG(chat_model, documents, k=5)                 # our (naive) hackathon baseline 

query = "What is Allplan?"

the_relevant_docs = rag.retrieve(question=query)

answer, relevant_docs = rag.answer(question=query)
answer, relevant_docs

('Allplan is a software that offers various tools to control how models and design elements appear on the screen. It is used for designing and modifying elements, including designing buildings (architecture) and creating key plans, general arrangement drawings, and reinforcement drawings (engineering).\n\nAllplan has documentation, including Help, Manuals, and Tutorials (Basics, Architecture, Engineering), and features such as libraries, smart symbols, SmartParts, PythonParts, and Allplan Visual Scripting. It supports data exchange, system administration, geodesy, presentation, and 3D modeling. Allplan also has a user interface and update settings, and is associated with CAD tutorials.',
 [Document(metadata={}, page_content='Installation, Basics Controlling what’s on your screen 105 Controlling what’s on your screen Allplan offers various tools you can use to control how your model and its design elements appear on the screen. Thus, you can always choose the tool best suited to the tas

In [8]:
# 2. Create the builder with the RAG instance
builder = eval.EvalDatasetBuilder(rag)

# 3. Build the evaluation dataset
evaluation_dataset = builder.build_evaluation_dataset(
    input_contexts=ground_truth_contexts,
    sample_queries=sample_queries,
    expected_responses=expected_responses,
)

Building test dataset: 100%|██████████| 100/100 [05:21<00:00,  3.22s/it]
2025-11-21 23:05:08,041 - ai_eval.resources.eval_dataset_builder - INFO - Evaluation dataset created successfully.
Building test dataset: 100%|██████████| 100/100 [05:21<00:00,  3.22s/it]
2025-11-21 23:05:08,041 - ai_eval.resources.eval_dataset_builder - INFO - Evaluation dataset created successfully.


In [9]:
from ai_eval.resources import deepeval_scorer as deep 

scorer = deep.DeepEvalScorer(evaluation_dataset)

results = scorer.calculate_scores()
print(results)

/Users/avosseler/Github/Team/tum-hackathon/.venv/lib/python3.12/site-packages/vertexai/_model_garden/_model_garden_models.py:278: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()
2025-11-21 23:05:24,350 - ai_eval.resources.get_models - INFO - Using chat model: gemini-2.5-flash
2025-11-21 23:05:24,352 - ai_eval.resources.get_models - INFO - Using judge model: gemini-2.5-flash
2025-11-21 23:05:24,352 - ai_eval.resources.get_models - INFO - Using embedding model: text-multilingual-embedding-002
2025-11-21 23:05:24,353 - ai_eval.resources.get_models - INFO - Models loaded successfully
2025-11-21 23:05:24,353 - ai_eval.resources.deepeval_scorer - INFO - Starting evaluation...
2025-11-21 23:05:24,350 - ai_eval.resources.get_models - INFO - Using chat model: gemini-2.5-flash
2025-11-21 23:05:24,352 - 

/Users/avosseler/Github/Team/tum-hackathon/.venv/lib/python3.12/site-packages/vertexai/_model_garden/_model_garden_models.py:278: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()
2025-11-21 23:05:24,350 - ai_eval.resources.get_models - INFO - Using chat model: gemini-2.5-flash
2025-11-21 23:05:24,352 - ai_eval.resources.get_models - INFO - Using judge model: gemini-2.5-flash
2025-11-21 23:05:24,352 - ai_eval.resources.get_models - INFO - Using embedding model: text-multilingual-embedding-002
2025-11-21 23:05:24,353 - ai_eval.resources.get_models - INFO - Models loaded successfully
2025-11-21 23:05:24,353 - ai_eval.resources.deepeval_scorer - INFO - Starting evaluation...
2025-11-21 23:05:24,350 - ai_eval.resources.get_models - INFO - Using chat model: gemini-2.5-flash
2025-11-21 23:05:24,352 - 

✨ You're running DeepEval's latest Answer Relevancy Metric! (using LangChain Model, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Faithfulness Metric! (using LangChain Model, strict=False, async_mode=True)...

✨ You're running DeepEval's latest Contextual Recall Metric! (using LangChain Model, strict=False, 
async_mode=True)...

...

Evaluating 100 test case(s) in parallel: |          |  0% (0/100) [Time Taken: 00:00, ?test case/s]/Users/avosseler/Github/Team/tum-hackathon/src/ai_eval/utils/utils.py:157: LangChainDeprecationWarning: The method `BaseChatModel.apredict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~ainvoke` instead.
  res = await chat_model.apredict(prompt)
/Users/avosseler/Github/Team/tum-hackathon/src/ai_eval/utils/utils.py:157: LangChainDeprecationWarning: The method `BaseChatModel.apredict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~ainvoke` instead.
  res = await chat_model.apredict(prompt)
I0000 00:00:1763762735.027039 6177869 ssl_transport_security.cc:1884] Handshake failed with error SSL_ERROR_SSL: error:1000007d:SSL routines:OPENSSL_internal:CERTIFICATE_VERIFY_FAILED: unable to get local issuer certificate
I0000 00:00:1763762735.027039 6177869 ssl_transport_security.cc:1884] Handshake failed with error SSL_ERROR_SSL: error:



Metrics Summary

  - ✅ Answer Relevancy (score: 1.0, threshold: 0.5, strict: False, evaluation model: LangChain Model, reason: The score is 1.00 because the output perfectly addresses the input without any irrelevant information. Excellent work!, error: None)
  - ✅ Faithfulness (score: 1.0, threshold: 0.7, strict: False, evaluation model: LangChain Model, reason: The score is 1.00 because there are no contradictions found, indicating excellent faithfulness!, error: None)
  - ✅ Contextual Recall (score: 1.0, threshold: 0.7, strict: False, evaluation model: LangChain Model, reason: The score is 1.00 because every sentence in the expected output is fully supported by the information in nodes 1 and 2 in retrieval context. Excellent work!, error: None)
  - ✅ Contextual Precision (score: 1.0, threshold: 0.7, strict: False, evaluation model: LangChain Model, reason: The score is 1.00 because all relevant nodes were ranked perfectly at the top!, error: None)

For test case:

  - input: What 

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

2025-11-21 23:08:07,870 - ai_eval.resources.deepeval_scorer - INFO - Evaluation completed successfully!


test_results=[TestResult(name='test_case_13', success=True, metrics_data=[MetricData(name='Answer Relevancy', threshold=0.5, success=True, score=1.0, reason='The score is 1.00 because the output perfectly addresses the input without any irrelevant information. Excellent work!', strict_mode=False, evaluation_model='LangChain Model', error=None, evaluation_cost=None, verbose_logs='Statements:\n[\n    "The two steps to define format properties based on layers in Allplan are:",\n    "Open the Format Definition tab.",\n    "Select one of the three options for matching the format properties of layers.",\n    "Switch to the Select Layer/Visibility tab.",\n    "Specify the format properties the program is to take from the layer.",\n    "pen thickness",\n    "line type",\n    "line color"\n] \n \nVerdicts:\n[\n    {\n        "verdict": "yes",\n        "reason": null\n    },\n    {\n        "verdict": "yes",\n        "reason": null\n    },\n    {\n        "verdict": "yes",\n        "reason": nul

In [10]:
scorer.get_overall_metrics()

{'Answer Relevancy': 0.9323052165699225,
 'Faithfulness': 0.9775,
 'Contextual Recall': 0.92,
 'Contextual Precision': 0.8888888888888888,
 'Average Performance': 0.9296735263647029}

In [11]:
scorer.get_summary(save_to_file=True)

2025-11-21 23:08:31,590 - ai_eval.resources.deepeval_scorer - INFO - Summary of test results generated successfully!
2025-11-21 23:08:31,604 - ai_eval.services.file - INFO - JSON Service Output to File: /Users/avosseler/Github/Team/tum-hackathon/data/deepeval_results.json
2025-11-21 23:08:31,604 - ai_eval.services.file - INFO - JSON Service Output to File: /Users/avosseler/Github/Team/tum-hackathon/data/deepeval_results.json


2025-11-21 23:08:31,590 - ai_eval.resources.deepeval_scorer - INFO - Summary of test results generated successfully!
2025-11-21 23:08:31,604 - ai_eval.services.file - INFO - JSON Service Output to File: /Users/avosseler/Github/Team/tum-hackathon/data/deepeval_results.json
2025-11-21 23:08:31,604 - ai_eval.services.file - INFO - JSON Service Output to File: /Users/avosseler/Github/Team/tum-hackathon/data/deepeval_results.json


{0: {'query': 'What are the two steps to define format properties based on layers in Allplan?',
  'actual_output': 'The two steps to define format properties based on layers in Allplan are:\n\n1.  Open the Format Definition tab and select one of the three options for matching the format properties of layers.\n2.  Switch to the Select Layer/Visibility tab and specify the format properties (pen thickness, line type and line color) the program is to take from the layer.',
  'expected_output': 'First, open the Format Definition tab and select an option for matching layer properties. Second, switch to the Select Layer /Visibility tab and specify the pen thickness, line type, and line color for the layer.',
  'actual_context': ['Installation, Basics Using format properties 121 Defining format properties based on layers In the Layer dialog box, you can specify that an element is to automatically assume the properties of the layer on which the element is drawn. This involves two steps: • Open 

2025-11-21 23:08:31,590 - ai_eval.resources.deepeval_scorer - INFO - Summary of test results generated successfully!
2025-11-21 23:08:31,604 - ai_eval.services.file - INFO - JSON Service Output to File: /Users/avosseler/Github/Team/tum-hackathon/data/deepeval_results.json
2025-11-21 23:08:31,604 - ai_eval.services.file - INFO - JSON Service Output to File: /Users/avosseler/Github/Team/tum-hackathon/data/deepeval_results.json


{0: {'query': 'What are the two steps to define format properties based on layers in Allplan?',
  'actual_output': 'The two steps to define format properties based on layers in Allplan are:\n\n1.  Open the Format Definition tab and select one of the three options for matching the format properties of layers.\n2.  Switch to the Select Layer/Visibility tab and specify the format properties (pen thickness, line type and line color) the program is to take from the layer.',
  'expected_output': 'First, open the Format Definition tab and select an option for matching layer properties. Second, switch to the Select Layer /Visibility tab and specify the pen thickness, line type, and line color for the layer.',
  'actual_context': ['Installation, Basics Using format properties 121 Defining format properties based on layers In the Layer dialog box, you can specify that an element is to automatically assume the properties of the layer on which the element is drawn. This involves two steps: • Open 

I0000 00:00:1763762948.150308 6177870 ssl_transport_security.cc:1884] Handshake failed with error SSL_ERROR_SSL: error:1000007d:SSL routines:OPENSSL_internal:CERTIFICATE_VERIFY_FAILED: unable to get local issuer certificate
I0000 00:00:1763763053.207286 6177866 ssl_transport_security.cc:1884] Handshake failed with error SSL_ERROR_SSL: error:1000007d:SSL routines:OPENSSL_internal:CERTIFICATE_VERIFY_FAILED: unable to get local issuer certificate
I0000 00:00:1763763053.207286 6177866 ssl_transport_security.cc:1884] Handshake failed with error SSL_ERROR_SSL: error:1000007d:SSL routines:OPENSSL_internal:CERTIFICATE_VERIFY_FAILED: unable to get local issuer certificate


In [ ]:
from ai_eval.services.file import JSONService
from ai_eval.config import global_config as glob

json = JSONService(path="simple_benchmark_tfidf_N100.json", root_path=glob.DATA_PKG_DIR, verbose=True)

json.doWrite(scorer.get_overall_metrics())

2025-11-21 23:12:26,126 - ai_eval.services.file - INFO - JSON Service Output to File: /Users/avosseler/Github/Team/tum-hackathon/data/simple_benchmark_tfidf_N100.json


2025-11-21 23:12:26,126 - ai_eval.services.file - INFO - JSON Service Output to File: /Users/avosseler/Github/Team/tum-hackathon/data/simple_benchmark_tfidf_N100.json


I0000 00:00:1763763173.284759 6177871 ssl_transport_security.cc:1884] Handshake failed with error SSL_ERROR_SSL: error:1000007d:SSL routines:OPENSSL_internal:CERTIFICATE_VERIFY_FAILED: unable to get local issuer certificate
I0000 00:00:1763763308.381261 6177870 ssl_transport_security.cc:1884] Handshake failed with error SSL_ERROR_SSL: error:1000007d:SSL routines:OPENSSL_internal:CERTIFICATE_VERIFY_FAILED: unable to get local issuer certificate
I0000 00:00:1763763308.381261 6177870 ssl_transport_security.cc:1884] Handshake failed with error SSL_ERROR_SSL: error:1000007d:SSL routines:OPENSSL_internal:CERTIFICATE_VERIFY_FAILED: unable to get local issuer certificate
I0000 00:00:1763763408.450223 6177869 ssl_transport_security.cc:1884] Handshake failed with error SSL_ERROR_SSL: error:1000007d:SSL routines:OPENSSL_internal:CERTIFICATE_VERIFY_FAILED: unable to get local issuer certificate
I0000 00:00:1763763548.549240 6177873 ssl_transport_security.cc:1884] Handshake failed with error SSL_ER